In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c crowdflower-search-relevance

100% 4.51M/4.51M [00:00<00:00, 9.71MB/s]

  0% 0.00/51.2k [00:00<?, ?B/s]
100% 51.2k/51.2k [00:00<00:00, 55.4MB/s]
  0% 0.00/1.87M [00:00<?, ?B/s]
100% 1.87M/1.87M [00:00<00:00, 125MB/s]


In [ ]:
!mkdir train

In [ ]:
! unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [ ]:
!mkdir test
! unzip test.csv.zip -d test

Archive:  test.csv.zip
  inflating: test/test.csv           


In [ ]:
! unzip sampleSubmission.csv.zip

Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    


In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("train/train.csv").fillna("")
test = pd.read_csv("test/test.csv").fillna("")

In [ ]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,0,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,3,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,3,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",3,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",1,0.471


In [ ]:
train['median_relevance'] = train['median_relevance'].apply(lambda x: x - 1)

In [ ]:
test.head()

,id,query,product_title,product_description
0,3,electric griddle,Star-Max 48 in Electric Griddle,
1,6,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,
2,9,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...
3,11,aveeno shampoo,AVEENO 10.5FLOZ NRSH SHINE SH,"Water, Ammonium Lauryl Sulfate, Dimethicone, S..."
4,12,flea and tick control for dogs,Merial Frontline Plus Flea and Tick Control fo...,


In [ ]:
len(train)

10158

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
#import nltk 
#from nltk.tokenize import WordPunctTokenizer

In [ ]:
#tokenizer = nltk.tokenize.WordPunctTokenizer()
#preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

In [ ]:
#train['pd'] = train['product_description'].apply(preprocess)

In [ ]:
train['next_sent'] = train['product_description'] + train['product_title']

In [ ]:
train['quer'] = train['query']

In [ ]:
test['next_sent'] = test['product_description'] + test['product_title']

In [ ]:
test['quer'] = test['query']

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10158 entries, 0 to 10157
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   query                10158 non-null  object 
 1   product_title        10158 non-null  object 
 2   product_description  10158 non-null  object 
 3   median_relevance     10158 non-null  int64  
 4   relevance_variance   10158 non-null  float64
 5   next_sent            10158 non-null  object 
 6   quer                 10158 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 555.6+ KB


In [ ]:
from collections import Counter

In [ ]:
Counter(train.median_relevance)

Counter({0: 774, 1: 1476, 2: 1737, 3: 6171})

In [ ]:
value_count = Counter(train.median_relevance)
class_weight_dict = {0:(1 / value_count[0])*(len(train))/4.0,
                     1:(1 / value_count[1])*(len(train))/4.0,
                     2:(1 / value_count[2])*(len(train))/4.0,
                     3:(1 / value_count[3])*(len(train))/4.0}
class_weight_dict

{0: 3.2810077519379846,
 1: 1.7205284552845528,
 2: 1.4620034542314335,
 3: 0.4115216334467671}

In [ ]:
!pip install transformers

In [ ]:
#Next sentence prediction problem, Using BertForNextSentencePrediction with Linear Layer on top

In [ ]:
from transformers import BertModel, BertTokenizer

In [ ]:
import torch.nn as nn
import transformers
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', use_fast = True)

In [ ]:
#model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
class NSP(nn.Module):
  def __init__(self):
    super(NSP, self).__init__()
    self.nsp = BertModel.from_pretrained('bert-base-uncased')
    self.drop = nn.Dropout(0.5)
    self.out = nn.Linear(self.nsp.config.hidden_size, 4)
    #self.softmax = nn.softmax()

  def forward(self, ids, mask, token_type_ids):
    output = self.nsp(input_ids = ids, attention_mask = mask, token_type_ids = token_type_ids)
    bo = self.drop(output.pooler_output)
    return self.out(bo)

class CFDataset(Dataset):
  def __init__(self, quer, next_sent, target, tokenizer, max_len):
    self.quer = quer
    self.next_sent = next_sent
    self.target = target
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.target)

  def __getitem__(self, item):
    quer = str(self.quer[item])
    next_sent = str(self.next_sent[item])

    inputs = tokenizer.encode_plus(
        quer,
        next_sent,
        truncation = True,
        padding = 'max_length',
        max_length = self.max_len
    )        

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        "ids" : torch.tensor(ids, dtype = torch.long),
        "mask" : torch.tensor(mask, dtype = torch.long),
        "token_type_ids" : torch.tensor(token_type_ids, dtype = torch.long),
        "targets" : torch.tensor(self.target[item], dtype = torch.long)
    }

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_valid = train_test_split(train, test_size=0.1, random_state=111, stratify = train.median_relevance)

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = CFDataset(
    quer=df.quer.to_numpy(),
    next_sent = df.next_sent.to_numpy(),
    target=df.median_relevance.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size
  )

In [ ]:
BATCH_SIZE = 16
MAX_LEN = 128
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_valid, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['ids', 'mask', 'token_type_ids', 'targets'])

In [ ]:
print(data['ids'].shape)
print(data['mask'].shape)
print(data['targets'].shape)
print(data['token_type_ids'].shape)

torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16])
torch.Size([16, 128])


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using:', device)

Using: cuda


In [ ]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13021         731        9409           1        2880       12062
Swap:             0           0           0


In [ ]:
model = NSP()
model = model.to(device)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss(weight=weights).to(device)

In [ ]:
weights = torch.Tensor([3.281, 1.705, 1.462, 0.412]).to(device)

In [ ]:
class_weight_dict.values()

dict_values([3.2810077519379846, 1.7205284552845528, 1.4620034542314335, 0.4115216334467671])

In [ ]:
import tqdm

In [ ]:
import numpy as np


In [ ]:
def train_epoch(
    model, 
    data_loader, 
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in tqdm.notebook.tqdm(data_loader):
    input_ids = d["ids"].to(device)
    attention_mask = d["mask"].to(device)
    token_type_ids = d["token_type_ids"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      ids=input_ids,
      mask=attention_mask,
      token_type_ids = token_type_ids
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["ids"].to(device)
      attention_mask = d["mask"].to(device)
      token_type_ids = d["token_type_ids"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        ids=input_ids,
        mask=attention_mask,
        token_type_ids = token_type_ids
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
from collections import defaultdict

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_valid)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------



Train loss 0.8387627644607654 accuracy 0.723911616714067
Val   loss 0.6830165316350758 accuracy 0.7942913385826772

Epoch 2/10
----------



Train loss 0.5946089163604942 accuracy 0.8143732224896084
Val   loss 0.687327251303941 accuracy 0.7883858267716536

Epoch 3/10
----------



Train loss 0.44851214228966435 accuracy 0.8685189236490921
Val   loss 0.8002899118000641 accuracy 0.8070866141732284

Epoch 4/10
----------



Train loss 0.3481597068908435 accuracy 0.9046160577554146
Val   loss 0.9288116142270155 accuracy 0.8139763779527559

Epoch 5/10
----------


...


Train loss 0.22463325614197557 accuracy 0.9443229052723693
Val   loss 1.149412574217422 accuracy 0.8070866141732284

Epoch 7/10
----------



Train loss 0.18035593225439145 accuracy 0.9525267993874426
Val   loss 1.2684909423223871 accuracy 0.8061023622047244

Epoch 8/10
----------



Train loss 0.15602284455820478 accuracy 0.9637934806388099
Val   loss 1.305273050773394 accuracy 0.8139763779527559

Epoch 9/10
----------



Train loss 0.1341358131554647 accuracy 0.9682782760883833
Val   loss 1.3821106079412857 accuracy 0.8218503937007874

Epoch 10/10
----------



Train loss 0.10603891128083524 accuracy 0.9748413913804419
Val   loss 1.4071766410852433 accuracy 0.8297244094488189

CPU times: user 26min 35s, sys: 14min 3s, total: 40min 38s
Wall time: 40min 54s
